In [27]:
import os
import shutil
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import errno
import stat

def handle_remove_readonly(func, path, exc):
    exc_value = exc[1]
    if exc_value.errno == errno.ENOENT:
        pass
    elif exc_value.errno == errno.EACCES:
        os.chmod(path, stat.S_IWUSR)
        func(path)
    else:
        raise

def decimal_to_custom_base(n):
    """Converts a decimal number to a base-25 string using a custom non-confusing character set."""
    characters = 'acdefhjkmnpqrtuvwxyz23478'
    base = len(characters)
    if n == 0:
        return characters[0]
    result = ""
    while n > 0:
        n, remainder = divmod(n, base)
        result = characters[remainder] + result
    return result

def human_readable_size(size):
    """Converts bytes to a more human-readable format."""
    if size < 1024:
        return f"{size} B"
    elif size < 1024 ** 2:
        return f"{size / 1024:.2f} KB"
    elif size < 1024 ** 3:
        return f"{size / (1024 ** 2):.2f} MB"
    else:
        return f"{size / (1024 ** 3):.2f} GB"

def resize(directory, grayscale=True):
    """Resizes all images in a directory to 256x256, converts them to grayscale, and renames them using a base-25 encoding."""
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    successful_files = set()
    failed_counter = 0

    target_dir = Path(directory)
    thumbnail_dir = target_dir / "thumbnail"
    if not target_dir.is_dir():
        tqdm.write(f"Directory does not exist: {directory}")
        return

    if thumbnail_dir.exists():
        tqdm.write(f"Deleting existing thumbnail directory: {thumbnail_dir}")
        shutil.rmtree(thumbnail_dir, onexc=handle_remove_readonly)
    thumbnail_dir.mkdir(parents=True, exist_ok=True)

    # Collect all images and calculate total size
    total_size = 0
    image_paths = []
    for root, dirs, files in os.walk(directory):
        for name in files:
            if any(name.lower().endswith(ext) for ext in image_extensions):
                path = Path(root) / name
                image_paths.append(path)

    progress = tqdm(image_paths, desc="Scanning folders")
    images = []
    for path in progress:
        file_size = path.stat().st_size
        total_size += file_size
        images.append(path)
        progress.set_description(f"Collected {len(images)} images, total size: {human_readable_size(total_size)}")

    tqdm.write(f"Found {len(images)} images, total size: {human_readable_size(total_size)}")

    # Resize and convert images
    pbar = tqdm(images, desc="Processing images")
    for index, source_path in enumerate(pbar):
        new_name = decimal_to_custom_base(index + 1)  # Adjusted counter to be zero-indexed
        target_path = thumbnail_dir / f"{new_name}.jpg"
        
        try:
            if grayscale:
                img = Image.open(source_path).convert('L')
            else:
                img = Image.open(source_path)
            img = img.resize((256, 256), Image.LANCZOS)
            img.save(target_path, format='JPEG', quality=85)  # Save in JPEG format
            successful_files.add(new_name + '.jpg')
        except Exception as e:
            pbar.write(f"Error processing file {source_path}: {e}")
            failed_counter += 1

    # Calculate total size of generated thumbnails
    thumbnail_size = sum(p.stat().st_size for p in thumbnail_dir.iterdir())
    tqdm.write(f"Processing complete. Successful operations: {len(successful_files)}, Failed operations: {failed_counter}")
    tqdm.write(f"Total size of thumbnails: {human_readable_size(thumbnail_size)}")
    if total_size > 0:  # Avoid division by zero
        percentage = (thumbnail_size / total_size) * 100
        tqdm.write(f"Thumbnails are {percentage:.2f}% the size of the original images.")

开始处理...
Directory does not exist: /path/to/your/directory


In [28]:
target_directory = "/Volumes/192.168.1.173/pic/今田美樱_939[2123_MB]"
# target_directory = "/Users/chenweichu/dev/data/test_副本"

tqdm.write("开始处理...")
resize(target_directory)

开始处理...


Collected 931 images, total size: 1.98 GB: 100%|██████████| 931/931 [00:03<00:00, 275.93it/s]   


Found 931 images, total size: 1.98 GB


Processing images: 100%|██████████| 931/931 [07:41<00:00,  2.02it/s]


Processing complete. Successful operations: 931, Failed operations: 0
Total size of thumbnails: 10.83 MB
Thumbnails are 0.53% the size of the original images.
